In [2]:
import pandas as pd
import numpy as np
from utils import extract_matrix

In [3]:
df = pd.read_json("data_no_bool.json").T

In [5]:
df.head()

,label,time_series
0,6,"[{'BallAcceleration': 1636.7987723122642, 'Dis..."
1,-1,"[{'BallAcceleration': 0.0, 'DistanceWall': 39...."
2,-1,"[{'BallAcceleration': 124246.29375405672, 'Dis..."
3,-1,"[{'BallAcceleration': -8210.634011562943, 'Dis..."
4,-1,"[{'BallAcceleration': 1197.5360615055833, 'Dis..."


In [6]:
df.time_series.apply(extract_matrix)

0      [[1636.7987723122642, 3498.01, 2012.98, 299.66...
1      [[0.0, 39.7800000000002, 1704.5, 193.421147499...
2      [[124246.29375405672, 35.48999999999978, 1450....
3      [[-8210.634011562943, 36.0, 1036.28, 579.43975...
4      [[1197.5360615055833, 3975.02, 2013.04, 142.93...
                             ...                        
293    [[0.0, 2499.8900000000003, 2012.99, 2269.85802...
294    [[-4128.513654621303, 62170.1262183052, 1216.1...
295    [[0.0, 1606.3200000000002, 1911.36, 3064.80797...
296    [[0.0, 0.0, 2053.85, 2012.99, 4797.23569253169...
297    [[0.0, 1937.17, 1898.67, 5470.009844067925, 22...
Name: time_series, Length: 298, dtype: object

In [7]:
df.time_series = df.time_series.apply(extract_matrix)
df['shape'] = df.time_series.apply(lambda x: x.shape)
df.head()

,label,time_series,shape
0,6,"[[1636.7987723122642, 3498.01, 2012.98, 299.66...","(32, 17)"
1,-1,"[[0.0, 39.7800000000002, 1704.5, 193.421147499...","(39, 17)"
2,-1,"[[124246.29375405672, 35.48999999999978, 1450....","(34, 17)"
3,-1,"[[-8210.634011562943, 36.0, 1036.28, 579.43975...","(23, 17)"
4,-1,"[[1197.5360615055833, 3975.02, 2013.04, 142.93...","(22, 17)"
